## Short guide to qtealeaves's QteaTensor.
QteaTensor is a class that we use to represent a nonsymmetric tensor in our library in version $\geq$ v1.0.0. It contains the wrappers for common numpy/cupy array subroutines + some other functions that are useful when handling the tensor networks. In this guide, we will go through how to initialize a QteaTensor and some commonly used functions.

The source code can be found in
``
qtealeaves/tensors/tensor.py
``, where you can find all the relevant documentation and available functions.

In [14]:
from qtealeaves.tensors import QteaTensor
from qtealeaves.convergence_parameters import TNConvergenceParameters
import numpy as np

### Initializing a tensor

In [15]:
# possibility 1 - choose from the default initialization options
qtensor = QteaTensor(
                links = (2, 2), # dimension of each leg
                ctrl="R",   # initialization; 'R' is for random
                device="cpu", # cpu or gpu (i.e. using numpy or cupy for operations)
                dtype = complex
                )

# accessing the tensor elements
print('qtensor :')
print(qtensor.elem)
print(qtensor.dtype)

qtensor :
[[0.39466325+0.30880968j 0.68547865+0.84312796j]
 [0.7112913 +0.27994013j 0.09490103+0.75061234j]]
complex128


In [16]:
# possibility 2 - initialize from a numpy array
elements = np.arange((4))
elements = np.reshape(elements, (2,2))

qtensor = QteaTensor.from_elem_array(tensor=elements, dtype=float)
# note: be careful when creating qtensors out of array of integers to specify the dtype.
# Otherwise the values of tensor elements are locked to only integer values

# accessing the tensor elements
print('qtensor :')
print(qtensor.elem)
print(qtensor.dtype)

qtensor :
[[0. 1.]
 [2. 3.]]
float64


### Common functions

Identity tensor and changing the tensor backend

In [17]:
# initialize the identity tensor (= NxN matrix) with the same tensor backend as the reference tensor
identity_tensor = qtensor.eye_like(link=3)
print('identity tensor elements:')
print(identity_tensor.elem)
print('device, dtype:')
print(identity_tensor.device, identity_tensor.dtype)

# nevertheless we can always convert the dtype and the device of the tensor
identity_tensor.convert(dtype=int, device='cpu')
print('\nnew device, new dtype:')
print(identity_tensor.device, identity_tensor.dtype)

identity tensor elements:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
device, dtype:
cpu float64

new device, new dtype:
cpu int64


Attaching/removing a dummy link

In [18]:
print('shape : ', qtensor.elem.shape)

# attach a link of dimension 1 to a given position
qtensor.attach_dummy_link(position=0)
qtensor.attach_dummy_link(position=2)
print('shape : ', qtensor.shape)

# remove a link of dimension 1 at certain position
qtensor.remove_dummy_link(position=2)
qtensor.remove_dummy_link(position=0)
print('shape : ', qtensor.shape)


shape :  (2, 2)
shape :  (1, 2, 1, 2)
shape :  (2, 2)


Fusing/splitting the tensor indices

In [19]:
# suppose we want to fuse the leg 0, 1, and 4 of the rank-5 tensor
qtensor = QteaTensor(
                links = (1, 2, 3, 4, 5), # dimension of each leg
                ctrl="R",
                device="cpu"
                )
print('Original shape = ', qtensor.shape)

# first transpose such that legs that we want to fuse come consecutively
qtensor.transpose_update(permutation=[0,1,4,2,3])
print('Shape after transpose = ', qtensor.shape)

# then fuse the first 3 legs
qtensor.fuse_links_update(fuse_low=0, fuse_high=2)
print('Shape after fusing = ', qtensor.shape)

# let's go back to the original tensor
qtensor.reshape_update((1,2,5,3,4))
qtensor.transpose_update(permutation=[0,1,3,4,2])
print('\nBack to original shape : ', qtensor.shape)

Original shape =  (1, 2, 3, 4, 5)
Shape after transpose =  (1, 2, 5, 3, 4)
Shape after fusing =  (10, 3, 4)

Back to original shape :  (1, 2, 3, 4, 5)


QR and SVD decompositions

In [20]:
# QR decomposition
q_tens, r_tens = qtensor.split_qr(legs_left=[0,2], legs_right=[1,3,4])

print('Shapes after QR: ', q_tens.shape, r_tens.shape)

# SVD decomposition
# must define the convergence parameters
conv_params = TNConvergenceParameters(max_bond_dimension=2, cut_ratio=1e-5)
s_tens, d_tens, singvals, singvals_cut = qtensor.split_svd(legs_left=[0,2], legs_right=[1,3,4],
                                            conv_params=conv_params)

print('Shapes after SVD: ', s_tens.shape, d_tens.shape, singvals.shape, singvals_cut.shape)

Shapes after QR:  (1, 3, 3) (3, 2, 4, 5)
Shapes after SVD:  (1, 3, 2) (2, 2, 4, 5) (2,) (1,)
